In [ ]:
## The pipeline to generate a recipe (.csv) file for building the cellPACK models.
## Testing: build a much smaller scale of cytoplasm model of E-coli that contains all the cytoplasm proteins but only 
## with, let's say, one percent of original copy numbers for each component. 
## 1. From the bigg model, check the compartmentalization of every reactions and gather the ones that occur in cytoplasm.
## 2. Use ssbio to map the protein, sequence, downloading pdb, etc.. information onto these reactions.
## 3. Other stuff like color..
## the thing is only metabolites are compartementalized, not reactions or genes. How to filter out the reactions in 
## ex

In [2]:
import cobra
from ssbio.pipeline.gempro import GEMPRO
import sys
import logging
from os.path import join
import pandas as pd
import requests
import pickle
from ssbio.core.io import load_pickle

In [3]:
iML1515_df=pd.read_csv('/Applications/labfile/senior_design/data/iML1515_GP.csv')
proteomics_df=pd.read_csv('/Applications/labfile/senior_design/data/ecoli_cyto_proteomics.csv')
localization_df=pd.read_csv('/Applications/labfile/senior_design/data/ecoli_localization.csv')

In [4]:
proteomics_df

,Uniprot_Accession,Copies_Cell_MG1655_LB,Copies_Cell_MG1655_Glucose,proteinMW,Mol_Cell_MG1655_LB,Mol_Cell_MG1655_Glucose,Mass_Cell_MG1655_LB,Mass_Cell_MG1655_Glucose,Gene
0,P0A8T7,4696.0,2270.0,155045.00800,7.800000e-21,3.770000e-21,1.209024,0.584430,rpoC
1,P0A8V2,5649.0,2849.0,150520.27580,9.380000e-21,4.730000e-21,1.411938,0.712093,rpoB
2,P36683,17855.0,9626.0,93420.94570,2.960000e-20,1.600000e-20,2.769831,1.493273,acnB
3,P15254,414.0,2422.0,141295.89840,6.870000e-22,4.020000e-21,0.097136,0.568267,purL
4,P09831,272.0,1815.0,163176.31530,4.520000e-22,3.010000e-21,0.073701,0.491794,gltB
5,P00968,241.0,4414.0,117749.33280,4.000000e-22,7.330000e-21,0.047122,0.863058,carB
6,P25665,749.0,37939.0,84602.80905,1.240000e-21,6.300000e-20,0.105224,5.329908,metE
7,P0A6F5,55698.0,22305.0,57274.67483,9.250000e-20,3.700000e-20,5.297260,2.121358,groL
8,P0A6Y8,42936.0,18133.0,69054.47444,7.130000e-20,3.010000e-20,4.923370,2.079269,dnaK
9,P0A705,3236.0,1545.0,97272.32602,5.370000e-21,2.570000e-21,0.522693,0.249555,infB


In [5]:
iML1515_list=[]
proteomics_dict={}
proteom_uniprot_list=proteomics_df['Uniprot_Accession'].tolist()
gene_list=[]
uniprot_list=[]
for i in range(len(iML1515_df.m_gene)):
    new_dict={}
    new_dict['gene_id']=iML1515_df.m_gene[i]
    new_dict['uniprot_id']=iML1515_df.seq_uniprot[i]
    new_dict['m_subsystem']=iML1515_df.m_subsystem[i]
    new_dict['seq_mapped_pdbs']=iML1515_df.seq_mapped_pdbs[i]
    new_dict['seq_file']=iML1515_df.seq_file[i]
    new_dict['struct_pdb']=iML1515_df.struct_pdb[i]
    new_dict['struct_is_experimental']=iML1515_df.struct_is_experimental[i]  
    gene_list.append(iML1515_df.m_gene[i])
    uniprot_list.append(iML1515_df.seq_uniprot[i])
    iML1515_list.append(new_dict)
for i in range(len(proteomics_df.Uniprot_Accession)):
    uniprotid=proteomics_df.Uniprot_Accession[i]
    proteomics_dict[uniprotid]={'Copies_LB':proteomics_df.Copies_Cell_MG1655_LB[i],
                                'Copies_Glucose':proteomics_df.Copies_Cell_MG1655_Glucose[i]}

In [6]:
for i in range(len(iML1515_list)):
    uniprotid=iML1515_list[i]['uniprot_id']
    if uniprotid in proteom_uniprot_list:
        iML1515_list[i].update(proteomics_dict[uniprotid])
    else:
        iML1515_list[i].update({'Copies_LB':'N/A','Copies_Glucose':'N/A'})

In [16]:
iML1515_list

[{'Copies_Glucose': 'N/A',
  'Copies_LB': 'N/A',
  'gene_id': 'b0241',
  'm_subsystem': 'Transport, Outer Membrane Porin',
  'seq_file': 'P02932.fasta',
  'seq_mapped_pdbs': '1PHO',
  'struct_is_experimental': False,
  'struct_pdb': nan,
  'uniprot_id': 'P02932'},
 {'Copies_Glucose': 'N/A',
  'Copies_LB': 'N/A',
  'gene_id': 'b1377',
  'm_subsystem': 'Transport, Outer Membrane Porin',
  'seq_file': 'P77747.fasta',
  'seq_mapped_pdbs': nan,
  'struct_is_experimental': False,
  'struct_pdb': nan,
  'uniprot_id': 'P77747'},
 {'Copies_Glucose': 25214.0,
  'Copies_LB': 18617.0,
  'gene_id': 'b0929',
  'm_subsystem': 'Transport, Outer Membrane Porin',
  'seq_file': 'P02931.fasta',
  'seq_mapped_pdbs': '4JFB;4GCS;2OMF;1GFP;4LSH;1MPF;1OPF;4GCQ;1HXX;2ZLD;3HWB;2ZFG;4GCP;4LSF;3POQ;3POX;4LSE;1GFO;1HXU;3HW9;3FYX;1GFQ;4LSI;1GFM;3K1B;1BT9;1GFN;3K19;4D5U;1HXT;3POU;3O0E',
  'struct_is_experimental': True,
  'struct_pdb': '2zfg',
  'uniprot_id': 'P02931'},
 {'Copies_Glucose': 20763.0,
  'Copies_LB': 567

In [17]:
iML1515_dict

NameError: name 'iML1515_dict' is not defined

In [10]:
uniprot_mapped_to_model=[]
uniprot_notmapped_to_model=[]
for i in proteom_uniprot_list:
    if i in uniprot_list:
        uniprot_mapped_to_model.append(i)
    else:
        uniprot_notmapped_to_model.append(i)

In [24]:
a=[]
uniprot_notmapped_list=[]
for i in uniprot_notmapped_to_model:
    if i not in a:
        a.append(i)
for i in a:
    new_dict={}
    new_dict['uniprot_notmapped']=i
    uniprot_notmapped_list.append(new_dict)
#not_mapped_df=pd.DataFrame(data=uniprot_notmapped_dict)
#not_mapped_df.to_csv('/Applications/labfile/senior_design/data/uniprotID_notmapped.csv')

In [25]:
uniprot_notmapped_list

[{'uniprot_notmapped': 'P0A8T7'},
 {'uniprot_notmapped': 'P0A8V2'},
 {'uniprot_notmapped': 'P0A6F5'},
 {'uniprot_notmapped': 'P0A6Y8'},
 {'uniprot_notmapped': 'P0A705'},
 {'uniprot_notmapped': 'P0CE47'},
 {'uniprot_notmapped': 'P0A6M8'},
 {'uniprot_notmapped': 'P00957'},
 {'uniprot_notmapped': 'P00961'},
 {'uniprot_notmapped': 'P07118'},
 {'uniprot_notmapped': 'P63284'},
 {'uniprot_notmapped': 'P0AG67'},
 {'uniprot_notmapped': 'P00956'},
 {'uniprot_notmapped': 'P10408'},
 {'uniprot_notmapped': 'P07813'},
 {'uniprot_notmapped': 'P0A850'},
 {'uniprot_notmapped': 'P21889'},
 {'uniprot_notmapped': 'P0A940'},
 {'uniprot_notmapped': 'P05055'},
 {'uniprot_notmapped': 'P39180'},
 {'uniprot_notmapped': 'P0A9M0'},
 {'uniprot_notmapped': 'P21513'},
 {'uniprot_notmapped': 'P0AES4'},
 {'uniprot_notmapped': 'P0A6Z3'},
 {'uniprot_notmapped': 'P0AES6'},
 {'uniprot_notmapped': 'P07395'},
 {'uniprot_notmapped': 'P16659'},
 {'uniprot_notmapped': 'P0A9W3'},
 {'uniprot_notmapped': 'P0A6P1'},
 {'uniprot_not

In [7]:
tmp = pd.DataFrame.from_dict(iML1515_list).drop('m_subsystem', 1).drop_duplicates().reset_index(drop=True)

In [8]:
tmp

,Copies_Glucose,Copies_LB,gene_id,seq_file,seq_mapped_pdbs,struct_is_experimental,struct_pdb,uniprot_id
0,N/A,N/A,b0241,P02932.fasta,1PHO,False,NaN,P02932
1,N/A,N/A,b1377,P77747.fasta,NaN,False,NaN,P77747
2,25214,18617,b0929,P02931.fasta,4JFB;4GCS;2OMF;1GFP;4LSH;1MPF;1OPF;4GCQ;1HXX;2...,True,2zfg,P02931
3,20763,56743,b2215,P06996.fasta,2J1N;2ZLE;2J4U;4A8D;3NB3,True,2j1n,P06996
4,1526,4760,b3011,Q46856.fasta,1OJ7;4QGS,True,4qgs,Q46856
5,1,5,b4033,P02916.fasta,3RLF;3PUY;3PUX;3PUZ;3FH6;3PV0;4JBW;4KHZ;4KI0;2...,True,3rlf,P02916
6,N/A,N/A,b4032,P68183.fasta,3RLF;3PUY;3PUX;3PUZ;3FH6;3PV0;4JBW;4KI0;2R6G;3...,True,3rlf,P68183
7,25,78,b4035,P68187.fasta,1Q12;3RLF;1Q1B;3PUX;3PUY;3PUZ;3FH6;1Q1E;3PV0;4...,True,3rlf,P68187
8,2059,9363,b4034,P0AEX9.fasta,1JVX;3MP6;2NVU;4B3N;4IFP;3IOR;3SEW;4KEG;1DMB;4...,True,3kjt,P0AEX9
9,301,2048,b4036,P02943.fasta,1MPM;1MPO;1MPQ;1MPN;1AF6;1MAL,True,1af6,P02943


In [9]:
tmp[tmp['Copies_LB'] == 'N/A']

,Copies_Glucose,Copies_LB,gene_id,seq_file,seq_mapped_pdbs,struct_is_experimental,struct_pdb,uniprot_id
0,N/A,N/A,b0241,P02932.fasta,1PHO,False,NaN,P02932
1,N/A,N/A,b1377,P77747.fasta,NaN,False,NaN,P77747
6,N/A,N/A,b4032,P68183.fasta,3RLF;3PUY;3PUX;3PUZ;3FH6;3PV0;4JBW;4KI0;2R6G;3...,True,3rlf,P68183
10,N/A,N/A,b3579,P37676.fasta,NaN,False,NaN,P37676
11,N/A,N/A,b3578,P37675.fasta,NaN,False,NaN,P37675
12,N/A,N/A,b3577,P37674.fasta,NaN,False,NaN,P37674
14,N/A,N/A,b2835,P39196.fasta,NaN,False,NaN,P39196
16,N/A,N/A,b4298,P39359.fasta,NaN,False,NaN,P39359
27,N/A,N/A,b0446,P46891.fasta,NaN,False,NaN,P46891
28,N/A,N/A,b2302,P77526.fasta,5HFK;3GX0,True,5hfk,P77526


In [27]:
localization_dict={}
null={}
for i in range(localization_df.shape[0]):
    uniprotid=localization_df.iloc[i,0]
    new_dict2={}
    for j in range(localization_df.shape[1]-1):
        new_dict={}
        position=localization_df.columns[j+1]
        new_dict2[position]=localization_df.iloc[i,j+1]
        new_dict.update(new_dict2)
    localization_dict[uniprotid]=new_dict
for i in range(localization_df.shape[1]-1):
    new_dict={}
    position=localization_df.columns[i+1]
    new_dict[position]='N/A'
    null.update(new_dict)

In [28]:
local_uniprot_list=localization_df.Uniprot_Accession.tolist()
for i in range(len(iML1515_list)):
    uniprotid=iML1515_list[i]['uniprot_id']
    if uniprotid in local_uniprot_list:
        iML1515_list[i].update(localization_dict[uniprotid])
    else:
        iML1515_list[i].update(null)

In [33]:
for i in range(len(uniprot_notmapped_list)):
    uniprotid=uniprot_notmapped_list[i]['uniprot_notmapped']
    if uniprotid in proteom_uniprot_list:
        uniprot_notmapped_list[i].update(proteomics_dict[uniprotid])
    if uniprotid in local_uniprot_list:
        uniprot_notmapped_list[i].update(localization_dict[uniprotid])
    if uniprotid not in proteom_uniprot_list:
        uniprot_notmapped_list[i].update({'Copies_LB':'N/A','Copies_Glucose':'N/A'})
    if uniprotid not in local_uniprot_list:
        uniprot_notmapped_list[i].update(null)

In [37]:
iML1515_df=pd.DataFrame.from_records(iML1515_list)
iML1515_df.to_csv('/Applications/labfile/senior_design/data/iML1515_recipe.csv')

In [34]:
notmapped_df=pd.DataFrame.from_records(uniprot_notmapped_list)
notmapped_df.to_csv('/Applications/labfile/senior_design/data/notmapped_proteomics.csv')